In [14]:
import glob
import os
import numpy as np
import pandas as pd

In [12]:
def normalize(x, mmin, mmax):
    return (x - mmin) / (mmax - mmin)


def scale_array(x, mmin=0, mmax=1, new_min=-1, new_max=1):
    x = normalize(x, mmin, mmax)
    return np.clip(x * (new_max - new_min) + new_min, new_min, new_max)

In [22]:
script_dir = os.getcwd()
project_dir = os.path.split(os.path.split(script_dir)[0])[0]

# driver = 'default_driver'
# train_data_path = os.path.join(project_dir, '/data/csv/{}/*.csv'.format(driver))
train_data_path = os.path.join(project_dir, 'data/csv')

training_files = glob.glob(train_data_path + '/bot*.csv')

In [23]:
first = True
for f in training_files:
    # read dataset
    train_ds = pd.read_csv(f, header=None)

    X = train_ds.iloc[:, :-4].values
    y = train_ds.iloc[:, -4:].values

    X_full = X if first else np.concatenate((X_full, X))
    y_full = y if first else np.concatenate((y_full, y))

    first = False

In [24]:
for i in np.arange(X_full.shape[1]):
    print('{} - min: {}; max: {}'.format(i, X_full[:, i].min(), X_full[:, i].max()))

0 - min: -15.848694444400001; max: 29.0788888889
1 - min: -20.7573055556; max: 17.720361111099997
2 - min: -3.13042; max: 3.13409
3 - min: 0.0; max: 6.0
4 - min: 0.0; max: 9714.08
5 - min: -46.0786; max: 88.0865
6 - min: -55.1592; max: 87.8446
7 - min: -46.0739; max: 278.655
8 - min: -50.94; max: 278.39
9 - min: -1.0; max: 200.0
10 - min: -1.0; max: 200.0
11 - min: -1.0; max: 200.0
12 - min: -1.0; max: 200.0
13 - min: -1.0; max: 200.0
14 - min: -1.0; max: 200.0
15 - min: -1.0; max: 200.0
16 - min: -1.0; max: 200.0
17 - min: -1.0; max: 200.0
18 - min: -1.0; max: 200.0
19 - min: -1.0; max: 200.0
20 - min: -1.0; max: 200.0
21 - min: -1.0; max: 200.0
22 - min: -1.0; max: 200.0
23 - min: -1.0; max: 200.0
24 - min: -1.0; max: 200.0
25 - min: -1.0; max: 200.0
26 - min: -1.0; max: 200.0
27 - min: -1.0; max: 200.0


In [28]:
for i in np.arange(y_full.shape[1]):
    print('{} - min: {}; max: {}'.format(i, y_full[:, i].min(), y_full[:, i].max()))

0 - min: -1.0; max: 7.0
1 - min: -7.593300427999999; max: 6.7815199999999995
2 - min: 0.0; max: 1.0
3 - min: 0.0; max: 0.0


In [108]:
# Min and max values for specific parameters
maxSpeedX = X_full[:, 0].max()
minSpeedX = X_full[:, 0].min()
maxSpeedY = X_full[:, 1].max()
minSpeedY = X_full[:, 1].min()
maxRPM = X_full[:, 4].max()
maxWheelSpin = X_full[:, 9:28].max()

param_dict = {
    'maxSpeedX': maxSpeedX,
    'minSpeedX': minSpeedX,
    'maxSpeedY': maxSpeedY,
    'minSpeedY': minSpeedY,
    'maxRPM': maxRPM,
    'maxWheelSpin': maxWheelSpin
}

X_train = np.zeros(X_full.shape)

X_train[:, 0] = scale_array(X_full[:, 0], minSpeedX, maxSpeedX)
X_train[:, 1] = scale_array(X_full[:, 1], minSpeedY, maxSpeedY)
X_train[:, 2] = scale_array(X_full[:, 2], -math.pi, math.pi)

gears = X_full[:, 3]
gears[gears > 6] = 6
gears[gears < -1] = -1
X_train[:, 3] = scale_array(gears, -1, 6)

X_train[:, 4] = scale_array(X_full[:, 4], 0, maxRPM)

for i in np.arange(5, 9):
    X_train[:, i] = scale_array(X_full[:, i], 0, maxWheelSpin[i])
for i in np.arange(9, 28):
    X_train[:, i] = scale_array(X_full[:, i], 0, 200)

# y_train = np.zeros(y_full.shape)

# # gear = range(-1, 6)
# gears = y_full[:, 0]
# gears[gears == 7] = 6
# y_train[:, 0] = normalize_to_int(gears, -1, 6)
# # steering = range(-1, 1)
# y_train[:, 1] = np.clip(y_full[:, 1], -1, 1)
# # accelerate-brake = range(-1, 1)
# accel_brake = y_full[:, 2] - y_full[:, 3]
# y_train[:, 2] = normalize_to_int(accel_brake, -1, 1)

# y_train = np.delete(y_train, 3, axis=1)

In [109]:
for i in np.arange(X_train.shape[1]):
    print('{} - min: {}; max: {}'.format(i+1, X_train[:, i].min(), X_train[:, i].max()))

1 - min: -1.0; max: 1.0
2 - min: -1.0; max: 1.0
3 - min: -0.9999832398418258; max: 0.9999705074463787
4 - min: -1.0; max: 1.0
5 - min: -1.0; max: 1.0
6 - min: -1.0; max: 1.0
7 - min: -1.0; max: 1.0
8 - min: -1.0; max: 1.0
9 - min: -1.0; max: 1.0
10 - min: -1.0; max: 1.0
11 - min: -1.0; max: 1.0
12 - min: -1.0; max: 1.0
13 - min: -1.0; max: 1.0
14 - min: -1.0; max: 1.0
15 - min: -1.0; max: 1.0
16 - min: -1.0; max: 1.0
17 - min: -1.0; max: 1.0
18 - min: -1.0; max: 1.0
19 - min: -1.0; max: 1.0
20 - min: -1.0; max: 1.0
21 - min: -1.0; max: 1.0
22 - min: -1.0; max: 1.0
23 - min: -1.0; max: 1.0
24 - min: -1.0; max: 1.0
25 - min: -1.0; max: 1.0
26 - min: -1.0; max: 1.0
27 - min: -1.0; max: 1.0
28 - min: -1.0; max: 1.0
